In [ ]:
#imports
import tensorflow as tf
import numpy as np
import time
from scipy.stats import qmc


import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams["font.family"] = "Times New Roman"
mpl.rcParams["font.size"] = 12
mpl.rcParams["text.usetex"] = True


In [ ]:
tf.keras.backend.set_floatx("float32")

In [ ]:
dim = 3
n_bc = dim * 2
n_data_per_bc = 8

engine = qmc.LatinHypercube(d=1)
data = np.zeros([n_bc, n_data_per_bc, dim+1])

for i, j in zip(range(n_bc), [+1, -1]*n_bc):
    
    if i % 2 == 0:
        for dd in range(dim):
            if i // 2 == dd:
                data[i,:, dd] = j
            else:
                data[i,:, dd] = (engine.random(n=n_data_per_bc)[:, 0] - 0.5) * 2
    elif i % 2 == 1:
        for dd in range(dim):
            if i // 2 == dd:
                data[i,:, dd] = j
            else:
                data[i,:, dd] = (engine.random(n=n_data_per_bc)[:, 0] - 0.5) * 2

for i in range(1,n_bc):
    data[i,:,dim] = 1.

In [ ]:
data = data.reshape(n_data_per_bc*n_bc, dim+1)

In [ ]:
x_d, y_d, z_d, t_d = map(lambda x: np.expand_dims(x, axis=1), 
                    [data[:, 0], data[:, 1], data[:, 2], data[:, 3]])

In [ ]:
# 
N_c = 10000
engine = qmc.LatinHypercube(d=3)
colloc = engine.random(n=N_c)
colloc = 2 * (colloc -0.5)

In [ ]:
x_c, y_c, z_c = map(lambda x: np.expand_dims(x, axis=1), 
               [colloc[:, 0], colloc[:, 1], colloc[:, 2]])




In [ ]:
# Transform to float32
x_c, y_c, z_c, x_d, y_d, z_d, t_d =map(lambda x: tf.convert_to_tensor(x,dtype=tf.float32),
                             [x_c, y_c, z_c, x_d, y_d, z_d, t_d])

In [ ]:
fig = plt.figure("", figsize=(6, 6), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# 设置标题
ax.set_title("Boundary Data points and Collocation points")

# 绘制散点图
ax.scatter(x_d, y_d, z_d, marker='x', c='k', label='BDP')
ax.scatter(x_c, y_c, z_c, s=.2, marker=".", c="r", label="CP")

# 显示图形
plt.legend()
plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0PINN_DATA.png", bbox_inches='tight')
plt.show()

In [ ]:
# 
### model builder function
def DNN_builder(in_shape=2, out_shape=1, n_hidden_layers=10, 
                neuron_per_layer=20, actfn="tanh"):
    # input layer
    input_layer = tf.keras.layers.Input(shape=(in_shape,))
    # hidden layers
    hidden = [tf.keras.layers.Dense(neuron_per_layer, activation=actfn)(input_layer)]
    for i in range(n_hidden_layers-1):
        new_layer = tf.keras.layers.Dense(neuron_per_layer,
                                          activation=actfn,
                                          activity_regularizer=None)(hidden[-1])
        hidden.append(new_layer)
    # output layer
    output_layer = tf.keras.layers.Dense(1, activation=None)(hidden[-1])
    # building the model
    name = f"DNN-{n_hidden_layers}"
    model = tf.keras.Model(input_layer, output_layer, name=name)
    return model

In [ ]:
import os

In [ ]:
#

layers = [1, 4, 8]
loss_dict = {}
train_time = {}

n = 32
X = np.linspace(-1, +1, n)
Y = np.linspace(-1, +1, n)
Z = np.linspace(-1, +1, n)
X0, Y0, Z0 = np.meshgrid(X, Y, Z)
X = X0.reshape([n*n*n, 1])
Y = Y0.reshape([n*n*n, 1])
Z = Z0.reshape([n*n*n, 1])
X_T = tf.convert_to_tensor(X)
Y_T = tf.convert_to_tensor(Y)
Z_T = tf.convert_to_tensor(Z)


for it, layer in enumerate(layers):
    os.mkdir(f"outputs3D{layer}")
    
    @tf.function
    def f(x, y, z):
        u0 = u(x, y, z)
        u_x = tf.gradients(u0, x)[0]
        u_y = tf.gradients(u0, y)[0]
        u_z = tf.gradients(u0, z)[0]
        
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]
        u_zz = tf.gradients(u_z, z)[0]
        F = u_xx + u_yy + u_zz
        return tf.reduce_mean(tf.square(F))
    
    @tf.function
    def u(x, y, z):
        u = model(tf.concat([x, y, z], axis=1))
        return u
    @tf.function
    def mse(y, y_):
        return tf.reduce_mean(tf.square(y-y_))

    
    tf.keras.backend.clear_session()
    model = DNN_builder(3, 1, layer, 20, "tanh")
    # model.summary()
    # tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, 
    #                           show_layer_names=True, show_dtype=True, 
    #                           show_layer_activations=True)
    
    loss = 0
    epochs = 1000
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)

    
    epoch = 0
    loss_values = np.array([])
    start = time.time()
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            T_ = u(x_d, y_d, z_d)
            l = mse(t_d, T_)
    
            L = f(x_c, y_c, z_c)
            loss = l+L
        g = tape.gradient(loss, model.trainable_weights)
        opt.apply_gradients(zip(g, model.trainable_weights))
        loss_values = np.append(loss_values, loss)
        if epoch % 20 == 0 or epoch == epochs-1:
            print(f"{epoch:5}, {loss.numpy():.3f}")
            S = u(X_T, Y_T, Z_T)*10.
            S = S.numpy().reshape(n, n, n)

            # for slice in range(n):
            #     file.write('\n\n')
            #     np.savetxt(f"outputs3D{layer}/outputPINN{epoch // 20+1}.dat", S[:,:,slice], fmt="%lf")
            
            with open(f"outputs3D{layer}/outputPINN{epoch // 20+1}.dat", 'w') as file:
                for slice in range(n):
                    for idx_x in range(n):
                        file.write("\n"+np.array2string(S[idx_x,:,slice], separator=' ', formatter={'float_kind':lambda x: "%.8f" % x})[1:-1].replace('\n', ''))
                    file.write("\n")

    end = time.time()
    print(f"\ncomputation time: {end-start:.3f}\n")

    loss_dict[layer] = loss_values
    train_time[layer] = end-start

In [ ]:
!rm -rf outputs3D1/

In [ ]:
fig = plt.figure(figsize=(6,5), dpi=150)
colors = {1: "b", 4:"b" ,8: "r"}
linestypes = {1:"-", 4: "-.", 8: "-"}
labels = {1: "PINN$3D$-$1$", 
          4:"PINN$3D$-$4$", 
          8:"PINN$3D$-$8$"} 
# xvalues = df[df.columns[0]]

for it in layers:
    # it = df.columns[i]
    plt.semilogy(loss_dict[it], 
             label=labels[it], 
             color=colors[it],
             linestyle=linestypes[it]
            )
# plt.xlim([0,1e3])
plt.ylabel("Estimated Mean Square Errors ($MSE$)")
plt.xlabel("Number of Iterations")
plt.grid()
plt.legend()
plt.title("Traning Results of Heat Equation with $d=3$ by PINN")
plt.savefig("/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0PINN3D.png", bbox_inches='tight')
plt.show()

In [ ]:
file

In [ ]:
with open('outputs3D1/outputPINN1.dat', 'a') as file:
    file.write("\n"+np.array2string(S[0,:,0], separator=' ', formatter={'float_kind':lambda x: "%.8f" % x})[1:-1].replace('\n', ''))

In [ ]:
(S[0,:,0].shape)

In [ ]:
np.array2string(S[0,:,0], separator=' ', formatter={'float_kind':lambda x: "%.8f" % x})[1:-1].replace('\n', '')

In [ ]:
# Create a 3D plots
fig = plt.figure(figsize=(15,5), dpi=200)

#
X = np.linspace(-1, +1, n)
Y = np.linspace(-1, +1, n)
Z = np.linspace(-1, +1, n)
X0, Y0, Z0 = np.meshgrid(X, Y, Z)
X = X0.reshape([n*n*n, 1])
Y = Y0.reshape([n*n*n, 1])
Z = Z0.reshape([n*n*n, 1])
X_T = tf.convert_to_tensor(X)
Y_T = tf.convert_to_tensor(Y)
Z_T = tf.convert_to_tensor(Z)


ax = fig.add_subplot(121, projection='3d')

S = u(X_T, Y_T, Z_T)
S = S.numpy().reshape(n, n, n)

# scatter = ax.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], c=u_ref, cmap='jet', marker='o',alpha=.5, s=2)
ax.scatter(X0, Y0, Z0, c=10.*S, cmap="jet")
# cbar = fig.colorbar(scatter, ax=ax, label='$u(1, x, y)$')
# plt.subplots_adjust(right=0.8)

# # Set labels and title
# ax.set_xlabel('X-axis')
# ax.set_ylabel('Y-axis')
# ax.set_title('Solution')

# plt.xlim([0,1])
# plt.ylim([0,1])

# ax2 = fig.add_subplot(122, projection='3d')
# cbar = fig.colorbar(scatter, ax=ax2, label='$u(1, x, y)$')
# scatter = ax2.scatter(Grid_World[:,0], Grid_World[:,1], Grid_World[:,2], alpha=.5, cmap='jet', marker='o', c = model(Grid_World, training=False),s=2)

# # Set labels and title
# ax2.set_xlabel('X-axis')
# ax2.set_ylabel('Y-axis')
# ax2.set_title('Prediction')

# plt.xlim([0,1])
# plt.ylim([0,1])

# plt.savefig(f"/Users/michaellea/001 - M.Sc_HPC/55640-Project/Seminar-2/Presentation/Slides/F0{d}.png", bbox_inches='tight')

plt.show()